In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using SymPy

In [51]:
# generate manipulator equation
# TODO: Switch to Symbolics.jl
sympy.init_printing(use_unicode=True)
@syms m1, m2, g, l1, l2
@syms q1()::real t
@syms q2()::real t
@syms q1d()::real t
@syms q2d()::real t
@syms q1dd()::real t
@syms q2dd()::real t

x1 = l1*sympy.cos(q1(t))
y1 = l1*sympy.sin(q1(t))
x2 = l1*sympy.cos(q1(t)) + l2*sympy.cos(q1(t) + q2(t))
y2 = l1*sympy.sin(q1(t)) + l2*sympy.sin(q1(t) + q2(t))

x1d = diff(x1, t)
y1d = diff(y1, t)
x2d = diff(x2, t)
y2d = diff(y2, t)

U = m1*g*y1 + m2*g*y2  # good
v1_squared = x1d^2 + y1d^2
v2_squared = x2d^2 + y2d^2
T = sympy.simplify(0.5*m1*v1_squared + 0.5*m2*v2_squared)

# Le Lagrangian
L = sympy.trigsimp(T - U)
L = L.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
L = L.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d

# Lagrange-Euler Equation
LE1 = diff(diff(L, q1d(t)), t) - diff(L, q1(t))  
LE2 = diff(diff(L, q2d(t)), t) - diff(L, q2(t))
LE = [LE1, LE2]
# LE = sympy.trigsimp(LE)

# subs first derivative
LE = LE.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
# subs second derivative
LE = LE.subs(sympy.Derivative(q1d(t), t), q1dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sympy.Derivative(q2d(t), t), q2dd(t))  # substitute d/dt q2d(t) with q2dd
LE = sympy.expand(sympy.simplify(LE))

# Generalized mass matrix
# M1 = sympy.Poly(LE[1], q1dd(t)).coeffs()
M11 = collect(LE[0], q1dd(t)).coeff(q1dd(t))
M12 = collect(LE[0], q2dd(t)).coeff(q2dd(t))
M21 = collect(LE[1], q1dd(t)).coeff(q1dd(t))
M22 = collect(LE[1], q2dd(t)).coeff(q2dd(t))
M = [[M11, M12] [M21, M22]]

# Coriolis Matrix
# BE CAREFUL about collecting coeffs -- check powers
#=
for n in 1:3
    C11 += collect(LE[0], q1d(t)).coeff(q1d(t), n)
    C12 += collect(LE[0], q2d(t)).coeff(q2d(t), n)
    C21 += collect(LE[1], q1d(t)).coeff(q1d(t), n)
    C22 += collect(LE[1], q2d(t)).coeff(q2d(t), n)
end
C = [C11*q1d(t) + C12*q2d(t), C21*q1d(t) + C22*q2d(t)]
=# 
# Gravity Matrix
G = LE
G = G.subs(q1d(t), 0)  # must remove q derivative terms manually
G = G.subs(q2d(t), 0)
G = G.subs(q1dd(t), 0)
G = G.subs(q2dd(t), 0)

# assume anything without qdd minus G is C
C = LE
C = C.subs(q1dd(t), 0)
C = C.subs(q2dd(t), 0)
C = C - G

LE

C = SymMatrix(PyObject Matrix([
[-2*l1*l2*m2*q1d(t)*q2d(t)*sin(q2(t)) - l1*l2*m2*q2d(t)**2*sin(q2(t))],
[                                       l1*l2*m2*q1d(t)**2*sin(q2(t))]]))


⎡                                                                             
⎢g⋅l₁⋅m₁⋅cos(q₁(t)) + g⋅l₁⋅m₂⋅cos(q₁(t)) + g⋅l₂⋅m₂⋅cos(q₁(t) + q₂(t)) + 1.0⋅l₁
⎢                                                                             
⎢                                                                             
⎣                                                                           g⋅

2                2                                                            
 ⋅m₁⋅q1dd(t) + l₁ ⋅m₂⋅q1dd(t) - 2⋅l₁⋅l₂⋅m₂⋅q1d(t)⋅q2d(t)⋅sin(q₂(t)) + 2⋅l₁⋅l₂⋅
                                                                              
                                       2                                      
l₂⋅m₂⋅cos(q₁(t) + q₂(t)) + l₁⋅l₂⋅m₂⋅q1d (t)⋅sin(q₂(t)) + l₁⋅l₂⋅m₂⋅q1dd(t)⋅cos(

                                    2                                         
m₂⋅q1dd(t)⋅cos(q₂(t)) - l₁⋅l₂⋅m₂⋅q2d (t)⋅sin(q₂(t)) + l₁⋅l₂⋅m₂⋅q2dd(t)⋅cos(q₂(
                                                  

In [52]:
M

2×2 Matrix{Sym}:
 1.0*l1^2*m1 + l1^2*m2 + 2*l1*l2*m2*cos(q2(t)) + l2^2*m2  …  l1*l2*m2*cos(q2(t)) + l2^2*m2
                           l1*l2*m2*cos(q2(t)) + l2^2*m2                           l2^2*m2

In [53]:
C

⎡                                                   2              ⎤
⎢-2⋅l₁⋅l₂⋅m₂⋅q1d(t)⋅q2d(t)⋅sin(q₂(t)) - l₁⋅l₂⋅m₂⋅q2d (t)⋅sin(q₂(t))⎥
⎢                                                                  ⎥
⎢                               2                                  ⎥
⎣                   l₁⋅l₂⋅m₂⋅q1d (t)⋅sin(q₂(t))                    ⎦

In [54]:
G

⎡g⋅l₁⋅m₁⋅cos(q₁(t)) + g⋅l₁⋅m₂⋅cos(q₁(t)) + g⋅l₂⋅m₂⋅cos(q₁(t) + q₂(t))⎤
⎢                                                                    ⎥
⎣                     g⋅l₂⋅m₂⋅cos(q₁(t) + q₂(t))                     ⎦

In [7]:
# https://github.com/JuliaPy/SymPy.jl/blob/master/src/lambdify.jl
# @syms q2
# M11 = M11.subs(q2(t), q2)
# gen_M11 = lambdify(M11) # , ("q1(t)", "q2(t)", "q1d(t)", "q2d(t)", "q1dd(t)", "q2dd(t)"))
# gen_M11(1.0)